In [1]:
!pip install transformers[sentencepiece]

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:

import pandas as pd

# Load your dataset
df = pd.read_csv("final_withaspects.csv")

# Iterate over rows and perform sentiment analysis
for index, row in df.iterrows():
  text = row['text']  # Assuming you have a 'text' column
  aspects = row['aspects']

  for aspect in aspects:
    # Tokenize input
    inputs = absa_tokenizer(text, aspect, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
      logits = absa_model(**inputs).logits

    # Extract predicted class
    predicted_class_idx = logits.argmax(-1).item()
    predicted_class = absa_model.config.id2label[predicted_class_idx]

    print(f"Text: {text}")
    print(f"Aspect: {aspect}")
    print(f"Sentiment: {predicted_class}")
    print("----")


In [3]:
sentence = "i loved everything and i love myself tooo"
print(f"Sentence: {sentence}")
print()

# ABSA of "food"
aspect = "food"
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
  print(f"Label {label}: {prob}")

Sentence: i loved everything and i love myself tooo

Sentiment of aspect 'food' is:
Label negative: 0.000622637802734971
Label neutral: 0.004710866138339043
Label positive: 0.9946664571762085


In [4]:
# ABSA of "service"
aspect = "service"
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
  print(f"Label {label}: {prob}")

Sentiment of aspect 'service' is:
Label negative: 0.0005104305455461144
Label neutral: 0.0031948674004524946
Label positive: 0.9962946772575378


In [5]:
# Overall sentiment of the sentence
sentiment = sentiment_model([sentence])[0]
print(f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")

Overall sentiment: positive with score 0.8659279942512512
